# Sample code for generating dummy data for monitoring

## Install packages

In [36]:
%pip install datasets ragas

/usr/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=992969) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install langfuse pandas weaviate-client jupyter openai ragas  # or
%pip install -r requirements.txt

## Setup env vars

1. Create a weaviate cluster at https://console.weaviate.cloud/create-cluster \
   [**Note**: ingested data in free account expires after 14 days]
2. *Wait* until the cluster is created
3. Click on the ![Weaviate code button](weaviate-connect.svg) button to show code to connect to the cluster
4. Add the REST endpoint into `WCS_URL` environment variable \
   `export WCS_URL="https://url.here"`
5. Also set the `WCS_API_KEY` environment variable to the admin API key. Click on the **API keys** button to reveal it. \
   `export WCS_API_KEY="api-key-here"`
6. Create new OpenAI API key at https://platform.openai.com/api-keys \
   `export OPENAI_API_KEY="key-here"`
7. Create new Langfuse API keys at https://cloud.langfuse.com/ (First create a project) \
   `export LANGFUSE_SECRET_KEY="secret-key"` \
   `export LANGFUSE_PUBLIC_KEY="public-key"` \
   `export LANGFUSE_HOST="https://cloud.langfuse.com"`

## Imports

In [1]:
# builtin packages
import os
import json
from textwrap import dedent

# 3rd party packages
import weaviate
import pandas as pd
from langfuse.openai import OpenAI
from langfuse import Langfuse
import langfuse

## Connect to weaviate cluster

In [2]:
# Set these environment variables
URL = os.getenv("WCS_URL")
APIKEY = os.getenv("WCS_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Connect to a WCS instance
wclient = weaviate.connect_to_wcs(
    cluster_url=URL,
    auth_credentials=weaviate.auth.AuthApiKey(APIKEY),
    headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
    }
)
lf_client = Langfuse()

## Terms
1. **Collection** \
   A collection is like a table
2. **Field** in a collection \
   A field is like a column of a table
3. **Vector** \
   A vector is an ordered array of numbers that represents a point in a multi-dimensional space. [Ref: ChatGPT]
4. **Embedding** \
   An embedding is the mapping of an object to a vector.

## Generating vector embeddings

The following things will happen when you run the code given below:
1. Read the `rag.csv` and do a basic cleanup
2. Create the collection with the specified schema and OpenAI's `text-embedding-3-large` embedding model. Click [here](https://platform.openai.com/docs/guides/embeddings) for more info on other OpenAI's models.
3. Import the records from the CSV to the newly created collection

In [3]:
def get_or_create_collection(collection_name: str, *, recreate: bool) -> weaviate.collections.collection.Collection:
    if recreate:
        wclient.collections.delete(collection_name)

    try:
        wclient.collections.list_all()[collection_name]
    except KeyError:
        pass
    else:
        return wclient.collections.get(collection_name)

    df = pd \
        .read_csv('rag.csv', dtype=str) \
        .dropna(ignore_index=True) \
        .drop_duplicates(subset=["question"], ignore_index=True) \
        .map(str.strip)
    
    collection = wclient.collections.create(
        name=collection_name,
        vectorizer_config=weaviate.classes.config.Configure.Vectorizer.text2vec_openai(model="text-embedding-3-large"),
        properties=[
            # Define db schema here
            weaviate.classes.config.Property(name="question", data_type=weaviate.classes.config.DataType.TEXT),
            weaviate.classes.config.Property(name="answer", data_type=weaviate.classes.config.DataType.TEXT),
        ],
    )

    # Import the records from the CSV to our weaviate collection
    with collection.batch.rate_limit(requests_per_minute=100) as batch:
        for _, row in df.iterrows():
            batch.add_object(properties=row.to_dict())

    return collection

In [4]:
collection = get_or_create_collection("Medqna", recreate=False)

## Get top 3 matches

In [10]:
message = "What food to avoid during pregnancy ?."
results = collection \
    .query \
    .near_text([message], limit=3) \
    .objects

In [11]:
results_new = (x.properties for x in results)
context = "\n".join(
    f'Question: "{x['question']}" , Answer: """{x['answer']}"""\n\n'
    for x in results_new
)

In [5]:
prompt = lf_client.get_prompt("medical-doctor").compile(context=context)

NameError: name 'context' is not defined

In [13]:
openai_client = OpenAI()

response = openai_client.chat.completions.create(
    # model="gpt-4-turbo-preview",
    model="gpt-4o",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user"  , "content": message},
    ],
    temperature=0.2,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

/home/compro/tmp/llm-sprint/.llm-venv/lib/python3.12/site-packages/langfuse/task_manager.py:315: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  event["timestamp"] = datetime.utcnow().replace(tzinfo=timezone.utc)


It's great that you're seeking information on how to maintain a healthy diet during pregnancy. Here's what you need to know about foods to avoid during this crucial time:

- Do not consume soft cheese, unpasteurized milk products, uncooked meat, fish, or eggs during pregnancy. These can contain harmful germs.
- Always wash fruits and vegetables before eating them. Cut fruits just before you plan to eat them.
- Avoid eating green papaya.
- Be cautious with fish that may contain high levels of mercury, as it can harm your baby. Limit the consumption of large fish known for high mercury content.
- Do not eat stale food, especially if it's more than a day old. If you have leftovers, store them in the fridge and reheat them thoroughly before eating.
- Avoid drugs, narcotics, and limit your intake of tea and coffee to keep your caffeine level low.

It's important to prioritize your health and the health of your baby during pregnancy. Always consult with a healthcare professional for personal

/home/compro/tmp/llm-sprint/.llm-venv/lib/python3.12/site-packages/langfuse/task_manager.py:315: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  event["timestamp"] = datetime.utcnow().replace(tzinfo=timezone.utc)


In [5]:
# import metrics
from ragas.metrics import faithfulness, answer_relevancy, context_precision
from ragas.metrics.critique import harmfulness
 
# metrics you chose
metrics = [faithfulness, answer_relevancy, harmfulness]

In [6]:
from ragas.run_config import RunConfig
from ragas.metrics.base import MetricWithLLM, MetricWithEmbeddings
 
 
# util function to init Ragas Metrics
def init_ragas_metrics(metrics, llm, embedding):
    for metric in metrics:
        if isinstance(metric, MetricWithLLM):
            metric.llm = llm
        if isinstance(metric, MetricWithEmbeddings):
            metric.embeddings = embedding
        run_config = RunConfig()
        metric.init(run_config)

In [7]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
 
# wrappers
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
 
llm = ChatOpenAI()
emb = OpenAIEmbeddings()
 
init_ragas_metrics(
    metrics,
    llm=LangchainLLMWrapper(llm),
    embedding=LangchainEmbeddingsWrapper(emb),
)
async def score_with_ragas(query, chunks, answer):
    scores = {}
    for m in metrics:
        print(f"calculating {m.name}")
        scores[m.name] = await m.ascore(
            row={"question": query, "contexts": chunks, "answer": answer}
        )
    return scores

In [8]:
row = next(pd \
        .read_csv('rag.csv', dtype=str) \
        .dropna(ignore_index=True) \
        .drop_duplicates(subset=["question"], ignore_index=True) \
        .map(str.strip)
           .iterrows())[1]

In [9]:
# start a new trace when you get a question
question = row['question']
trace = lf_client.trace(name = "rag")
 
# retrieve the relevant chunks
# chunks = get_similar_chunks(question)
contexts = row['answer']
# pass it as span
trace.span(
    name = "retrieval", input={'question': question}, output={'contexts': contexts}
)
 
# use llm to generate a answer with the chunks
# answer = get_response_from_llm(question, chunks)
answer = row['answer']
trace.span(
    name = "generation", input={'question': question, 'contexts': contexts}, output={'answer': answer}
)
 
# compute scores for the question, context, answer tuple
ragas_scores = await score_with_ragas(question, contexts, answer)

# send the scores
for m in metrics:
    trace.score(name=m.name, value=ragas_scores[m.name])

/home/compro/tmp/llm-sprint/.llm-venv/lib/python3.12/site-packages/langfuse/task_manager.py:315: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  event["timestamp"] = datetime.utcnow().replace(tzinfo=timezone.utc)
/home/compro/tmp/llm-sprint/.llm-venv/lib/python3.12/site-packages/langfuse/task_manager.py:315: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  event["timestamp"] = datetime.utcnow().replace(tzinfo=timezone.utc)


calculating faithfulness


/home/compro/tmp/llm-sprint/.llm-venv/lib/python3.12/site-packages/langfuse/task_manager.py:315: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  event["timestamp"] = datetime.utcnow().replace(tzinfo=timezone.utc)
/home/compro/tmp/llm-sprint/.llm-venv/lib/python3.12/site-packages/langfuse/task_manager.py:315: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  event["timestamp"] = datetime.utcnow().replace(tzinfo=timezone.utc)


calculating answer_relevancy


/home/compro/tmp/llm-sprint/.llm-venv/lib/python3.12/site-packages/langfuse/task_manager.py:315: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  event["timestamp"] = datetime.utcnow().replace(tzinfo=timezone.utc)


calculating harmfulness


/home/compro/tmp/llm-sprint/.llm-venv/lib/python3.12/site-packages/langfuse/task_manager.py:315: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  event["timestamp"] = datetime.utcnow().replace(tzinfo=timezone.utc)
